# PrioritizeMe

Aplikacja do priorytetowego planowania zadań to narzędzie, które pomoże efektywnie zarządzać czasem i zadaniami. Dzięki niej możesz tworzyć zadania, przypisywać im tytuły oraz opisy, a następnie nadawać im priorytety w zależności od ich ważności. 

Aplikacja wykorzystuje bazę danych `ROOM` do przechowywania wszystkich zadań, umożliwia podstawowe operacje *CRUD* (*Create-Remove-Update-Delete*), oraz ich filtrowanie i sortowanie.

<table><tr><td><img src="https://media2.giphy.com/media/v1.Y2lkPTc5MGI3NjExcnFqOGtlZmZkZXc0Mzg2OTJnbHYxcTZhNGZ5M29xMGV0bjNpODRveiZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/c56sRiyAvvGStsM7aA/giphy.gif" width="200" /></td><td><img src="https://media0.giphy.com/media/v1.Y2lkPTc5MGI3NjExOGQzd25qaDJmcWtoZXJzcmoxNDk3NnhjOWU4dnNyNG5lemJwdGtqaiZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/jS5GjdH5YAFhKu2w7l/giphy.gif" width="200" /></td><td><img src="https://media2.giphy.com/media/v1.Y2lkPTc5MGI3NjExdmN0aTdnZDBlaWRpMjJncXRpN253azZsM2syNWZyZjVsZ3I5NXp5MSZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/tYfIxGuV0ox7rDXIMM/giphy.gif" width="200" /></td></tr></table>

- Aplikacja zaimplementowana zgodnie ze wzoccem MVVM wraz z repozytorium
- `StateFlow` został wykorzystany do łatwego zarządzania bieżącym stanem interfejsu użytkownika
- Baza danych `Room` została wykorzystana do przechowywania zadań, oraz jako *SSOT* (*Single Source Of Truth*)
- Aplikacja zawiera trzy ekrany 
    - na głównym ekranie wyświetla listę zadań, umożliwia usunięcie, oznaczenie zadania jako wykonanego, oraz szybką zmianę nadanego priorytetu
    - ekran dodania zadania
    - ekran aktualizacji zadania, który umożliwia również usunięcie
- Aplikacja wykorzystuje `Jetpack Navigation` w celu utworzenia nawigacji w aplikacji
- `ViewModel` został umieszczony w głównek aktywności w celu uzyskania współdzielenia jego instancji przez wszystkie fragmenty


Dodajmy wymagane zależności do projektu

```verbatim
>>> Build.gradle(Project)
```

```kotlin
// Top-level build file where you can add configuration options common to all sub-projects/modules.
buildscript { // przed blokiem plugins
    repositories {
        google()
    }
    dependencies {
        classpath ("androidx.navigation:navigation-safe-args-gradle-plugin:2.5.3")
    }
}
plugins {
    id("com.android.application") version "8.1.0" apply false
    id("org.jetbrains.kotlin.android") version "1.8.0" apply false
}
```

```verbatim
>>> Build.gradle(Module)
```

```kotlin
plugins {
    id("com.android.application")
    id("org.jetbrains.kotlin.android")
    id("kotlin-kapt")
    id("androidx.navigation.safeargs.kotlin")
}
android {
    ...
    buildFeatures {
        viewBinding = true
    }
    compileOptions {
        sourceCompatibility = JavaVersion.VERSION_17
        targetCompatibility = JavaVersion.VERSION_17
    }
    kotlinOptions {
        jvmTarget = "17"
    }
}
dependencies {

    implementation("androidx.room:room-ktx:2.5.2")
    annotationProcessor("androidx.room:room-compiler:2.5.2")
    implementation("androidx.room:room-ktx:2.5.2")
    kapt("androidx.room:room-compiler:2.5.2")

    implementation ("androidx.navigation:navigation-fragment:2.5.3")
    implementation ("androidx.navigation:navigation-ui:2.5.3")

    // ViewModel
    implementation ("androidx.lifecycle:lifecycle-viewmodel-ktx:2.6.1")
    // LiveData
    implementation ("androidx.lifecycle:lifecycle-livedata-ktx:2.6.1")
    //Fragment
    implementation ("androidx.fragment:fragment-ktx:1.6.1")
    ...
}
```